In [8]:
import numpy as np
import math
import random
import os

In [9]:
class Fullerene:
    class BrennerPotential:
        def __init__(self, size, limit_bonds=False, xi=10.0, R0=1.315, R1=1.70, R2=2.00, De=6.325, S=1.29, lambda_=1.5, delta=0.80469, a0=0.011304, c0=19, d0=2.5):
            self.n = size
            self.limit_bonds = limit_bonds
            self.xi = xi
            self.R0 = R0
            self.R1 = R1
            self.R2 = R2
            self.De = De
            self.S = S
            self.lambda_ = lambda_
            self.delta = delta
            self.a0 = a0
            self.c0 = c0
            self.d0 = d0
            self.V_coeff = De / (S - 1.0)
            self.g_prep = 1 + c0 * c0 / (d0 * d0)
            self.r = np.zeros((size, size))
            self.f = np.zeros((size, size))
            self.V_R = np.zeros((size, size))
            self.V_A = np.zeros((size, size))
            self.B_avg = np.zeros((size, size))
            self.V = np.zeros(size)

        def __call__(self, atoms):
            self.fill_r(atoms)
            self.fill_f_VR_VA()
            self.fill_B_avg(atoms)
            self.fill_V()
            return 0.5 * np.sum(self.V)

        def fill_r(self, atoms):
            for i in range(self.n):
                for j in range(i + 1, self.n):
                    self.r[i, j] = self.r[j, i] = calc_r(i, j, atoms)

        def fill_f_VR_VA(self):
            for i in range(self.n):
                for j in range(self.n):
                    r_ij = self.r[i, j]

                    if r_ij > self.R2 or i == j:
                        self.f[i, j] = self.f[j, i] = 0.0
                        self.V_R[i, j] = self.V_R[j, i] = 0.0
                        self.V_A[i, j] = self.V_A[j, i] = 0.0
                        continue

                    if r_ij <= self.R1:
                        self.f[i, j] = self.f[j, i] = 1.0
                    else:
                        self.f[i, j] = self.f[j, i] = 0.5 * (1.0 + np.cos(math.pi * (r_ij - self.R1) / (self.R2 - self.R1)))

                    arg = self.lambda_ * (r_ij - self.R0)
                    self.V_R[i, j] = self.V_R[j, i] = self.V_coeff * np.exp(-np.sqrt(2.0 * self.S) * arg)
                    self.V_A[i, j] = self.V_A[j, i] = self.V_coeff * self.S * np.exp(-np.sqrt(2.0 / self.S) * arg)

        def fill_B_avg(self, atoms):
            for i in range(self.n):
                for j in range(i + 1, self.n):
                    r_ij = self.r[i, j]

                    if r_ij > self.R2 or i == j:
                        self.B_avg[i, j] = self.B_avg[j, i] = 0.0
                        continue

                    vr_ij = atoms[j] - atoms[i]
                    vr_ji = -vr_ij

                    xi_ij = 0.0
                    xi_ji = 0.0

                    for k in range(self.n):
                        r_ik = self.r[i, k]

                        if r_ik > self.R2 or k == i or k == j:
                            continue

                        cos_theta_ijk = np.dot(vr_ij, atoms[k] - atoms[i]) / r_ij / r_ik

                        if self.limit_bonds and cos_theta_ijk > 0.0:
                            xi_ij = self.xi
                            break

                        xi_ij += self.calc_f(r_ij) * self.a0 * (self.g_prep - self.c0 * self.c0 / (self.d0 * self.d0 + (1.0 + cos_theta_ijk) * (1.0 + cos_theta_ijk)))

                    for k in range(self.n):
                        r_jk = self.r[j, k]

                        if r_jk > self.R2 or k == i or k == j:
                            continue

                        cos_theta_jik = np.dot(vr_ji, atoms[k] - atoms[j]) / r_ij / r_jk

                        if self.limit_bonds and cos_theta_jik > 0.0:
                            xi_ji += self.xi
                        else:
                            xi_ji += self.calc_f(r_jk) * self.a0 * (self.g_prep - self.c0 * self.c0 / (self.d0 * self.d0 + (1.0 + cos_theta_jik) * (1.0 + cos_theta_jik)))

                    self.B_avg[i, j] = self.B_avg[j, i] = 0.5 * (math.pow(1.0 + xi_ij, -self.delta) + math.pow(1.0 + xi_ji, -self.delta))

        def fill_V(self):
            self.V.fill(0.0)

            for i in range(self.n):
                for j in range(self.n):
                    if self.r[i, j] > self.R2 or i == j:
                        continue

                    self.V[i] += self.f[i, j] * (self.V_R[i, j] - self.B_avg[i, j] * self.V_A[i, j])

        def calc_f(self, r_ij):
            if r_ij <= self.R1:
                return 1.0
            elif r_ij <= self.R2:
                return 0.5 * (1.0 + np.cos(math.pi * (r_ij - self.R1) / (self.R2 - self.R1)))
            else:
                return 0.0

    def __init__(self, size, r):
        self.size = size
        self.r = r
        self.atoms = self.generate_initial_positions(size, r)
        self.atoms_spherical = np.array([self.to_spherical(atom) for atom in self.atoms])
        self.BP = self.BrennerPotential(size)

    def generate_initial_positions(self, size, r):
        # Tu można dodać kod generujący początkowe położenia atomów
        pass

    def from_file(self, file):
        # Tu można dodać kod wczytujący położenia atomów z pliku
        pass

    def to_cartesian(self, atom_spherical):
        r, phi, theta = atom_spherical
        x = r * math.sin(theta) * math.cos(phi)
        y = r * math.sin(theta) * math.sin(phi)
        z = r * math.cos(theta)
        return np.array([x, y, z])

    def to_spherical(self, atom_cartesian):
        x, y, z = atom_cartesian
        r = math.sqrt(x * x + y * y + z * z)
        phi = math.atan2(y, x)
        theta = math.acos(z / r)
        return np.array([r, phi, theta])

    def try_shifting(self, beta, w_r, w_phi, w_theta, W_all, N):
        for n in range(N):
            index = random.randint(0, self.BP.n - 1)
            atom_old = self.atoms_spherical[index].copy()
            atom_new = atom_old.copy()

            for i in range(3):
                if i == 0:
                    atom_new[i] += random.uniform(-w_r, w_r)
                else:
                    atom_new[i] += random.uniform(-w_phi, w_phi)

            atom_new[2] = max(0.0, min(math.pi, atom_new[2]))

            if random.uniform(0, 1) < math.exp(-beta * (self.BP(self.atoms) - self.BP(self.atoms_spherical))):
                self.atoms[index] = self.to_cartesian(atom_new)
                self.atoms_spherical[index] = atom_new

    def get_energy(self):
        return self.BP(self.atoms)

    def write_positions(self, output_file):
        with open(output_file, 'w') as file:
            for atom in self.atoms:
                file.write(f'{atom[0]} {atom[1]} {atom[2]}\n')

def calc_r(i, j, atoms):
    return np.linalg.norm(atoms[j] - atoms[i])

def main():
    size = 60
    r = 1.0

    fullerene = Fullerene(size, r)
    beta_min = 0.001
    beta_max = 10.0
    w_r = 0.1
    w_phi = 0.1
    w_theta = 0.1
    W_all = 1000.0
    N = 100000

    for beta in np.linspace(beta_min, beta_max, num=100):
        fullerene.try_shifting(beta, w_r, w_phi, w_theta, W_all, N)

    energy = fullerene.get_energy()
    fullerene.write_positions('positions.txt')

if __name__ == "__main__":
    main()


TypeError: 'NoneType' object is not iterable